# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

We made a python package just for this course: lucem_illud. If you haven't installed this package, you should run the following code first. You don't need to install the package later; all you need to do is just to import the package with: import lucem_illud. 

In [1]:
!pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git
#installing lucem_illud package

  Cloning git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to /private/var/folders/qd/v0lr79px31q985c3rq2pm9zh0000gn/T/pip-req-build-c5mxpmev


If you're not familiar with jupyter notebook, you may wonder what the exclamation mark(!) at the beginning of the command does (or even what pip means). The exclamation mark enables us to execute Terminal commands in the notebook cells. 

In [2]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [3]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [4]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [5]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"X-uEKwpAIIQAAy9k7eAAAABS","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":991020438,"wgRevisionId":991020438,"wgArticleId":473317,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Articles needing expert attention from April 2008","All articles needing expert attention","Sociology ar

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

But before we proceed to Beautiful Soup, a digression about Python syntax, especially about objects and functions.
For those who are not familiar with the syntax of python (or, if you're familiar with R programming), you might wonder what requests.get or wikiContentRequest.text mean. To understand this, you need to first understand what objects are. You may have heard that Python is an object oriented programming language (unlike the procedure oriented programming language, an example of which is R). Object is a set of variables (or, data) and functions into which you pass your data. So, in object oriented programming languages, like python, variables and functions are bunleded into objects.

For example, let's look at wikiContentRequest. We use dir() function, which returns the list of attributes and functions of objects.

In [6]:
 dir(wikiContentRequest)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

There's 'text' here. We used 'wikiContentRequest.text' to access 'text.' In other words, we use .(dot notation) to access functions from objects. wikiContentRequest has a set of functions, as shown above, and we used 'wikiContentRequest.text' to access one of them. By the way, dot notations do not necessarily refer to functions--it refers to anything that the entity contains. 



Moving on to the next step: BeautifulSoup, a Python library which extracts data from HTML and XML, and transforms HTML files into Python objects.

In [7]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:1000])





Content analysis - Wikipedia
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"X-uEKwpAIIQAAy9k7eAAAABS","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":991020438,"wgRevisionId":991020438,"wgArticleId":473317,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Articles needing expert attention from April 2008","All articles needing expert attention","Sociology articles needing expert attention","Media articles needing expert attention","Articles prone to spam from October 2

This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [8]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

open() is a function which literally opens and returns the file. This function has multiple modes, and, here, we used mode as 'w', which means: open a file for writing. And then, we use 'write' function to write on the empty file (content_analysis_save) that we created using open(content_analysis_save, mode='w', encoding='utf-8').} What did we write on this file? The text we got from wikiContentRequest.text

Now let's open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag. 

In [9]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Content analysis is the study of documents and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.[1] One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in content analysis to auto

Another excursion for those who are not familiar with programming: for loop. For loop is used to iterate over a sequence. "ContentPTags" contains multiple paragraphs, each of which starts and ends with `<p>`. What the "for pTag in contentPtags[:3]" does here is: find each paragraph in contentPTags, which, here, we limited to the first three using contentPtags[:3], and then print each paragraph. So, we have three paragraphs. By the way, you can insert `<p>` in juputer notebook!

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [10]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Content analysis is the study of documents and...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3   Content analysis is best understood as a broad...
4   The simplest and most objective form of conten...
5   A further step in analysis is the distinction ...
6   Quantitative content analysis highlights frequ...
7   Siegfried Kracauer provides a critique of quan...
8   More generally, content analysis is research u...
9   By having contents of communication available ...
10  Computer-assisted analysis can help with large...
11  Robert Weber notes: "To make valid inferences ...
12  There are five types of texts in content analy...
13  Over the years, content analysis has been appl...
14  In recent times, particularly with the advent ...
15  Quantitative content analysis has enjoyed a re...
16  Content analysis can also be described as stud...
17  Manifest content is read

Since we learned how to do for loop, you might get what we did here: using contentParagraphs = [], we made an empty list; and then, for each paragraph in contentPTags, we substituted every [\d+\] with '', i.e., removed every [\d+\], and then appended each paragraph (now without [\d+\]) to the empty list. As we can see, we have a dataframe, each row of which is each paragraph of contentPTags, without reference indicators. 

By the way, what does [\d+\] mean? If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

Now we have a `DataFrame` containing all relevant text from the page ready to be processed

In [11]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<_sre.SRE_Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [12]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [13]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [14]:
# Links and file related to user-generated content
wikipedia_UGC = 'https://en.wikipedia.org/wiki/User-generated_content'
UGC_save = 'wikipedia_ugc.html'

In [15]:
requests.get(wikipedia_UGC)

<Response [200]>

In [16]:
# Parse the website content
wikiUGCRequest = requests.get(wikipedia_UGC)
wikiUGCSoup = bs4.BeautifulSoup(wikiUGCRequest.text, 'html.parser')
print(wikiUGCSoup.text[500:1000])

ated_content","wgTitle":"User-generated content","wgCurRevisionId":1000294312,"wgRevisionId":1000294312,"wgArticleId":3878926,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: location","CS1 maint: others","CS1 maint: multiple names: authors list","CS1 errors: missing periodical","Articles with short description","Short description matches Wikidata","Use dmy dates from September 2012","All articles with unsourced statements",



In [17]:
# Extract the text
with open(UGC_save, mode='w', encoding='utf-8') as f:
    f.write(wikiUGCRequest.text)

In [18]:
# Find all paragraphs in the website
UGCPTags = wikiUGCSoup.findAll('p')
for pTag in UGCPTags[:3]:
    print(pTag.text)



User-generated content (UGC), alternatively known as user-created content (UCC), is any form of content, such as images, videos, text, and audio, that has been posted by users on online platforms such as social media and wikis. It is a product consumers create to disseminate information about online products or the firms that market them.[1][2]

User-generated content is used for a wide range of applications, including problem processing, news, entertainment, advertising, gossip, research and many more. It is an example of the democratization of content production and the flattening of traditional media hierarchies. The BBC adopted a user-generated content platform for its websites in 2005, and TIME Magazine named "You" as the Person of the Year in 2006, referring to the rise in the production of UGC on Web 2.0 platforms.[3][4] CNN also developed a similar user-generated content platform, known as iReport.[5] There are other examples of news channels implementing similar protocols, e

In [19]:
UGCParagraphs = []
for pTag in UGCPTags:
    pTag_text = re.sub(r'\[\d+\]', '', pTag.text)
    pTag_text = re.sub(r'[&.,;:"]', '', pTag_text)
    pTag_text = re.sub(r'\n', '', pTag_text)
    pTag_text = re.sub(r'\'s', '', pTag_text)
    pTag_text = re.sub(r'[\'\/#!%\^\*{}=\-_`~()]', '', pTag_text)
    if pTag_text:
        UGCParagraphs.append(pTag_text)

The above code is used to remove trivial contents from the text using regular expressions. It removes any white spaces, punctuations from the texts to better prepare scraped text for further analysis. 

In [20]:
# Convert to a DataFrame
UGCParagraphsDF = pandas.DataFrame({'paragraph-text' : UGCParagraphs})
UGCParagraphsDF

,paragraph-text
0,Usergenerated content UGC alternatively known ...
1,Usergenerated content is used for a wide range...
2,The advent of usergenerated content marked a s...
3,The role of the passive audience therefore has...
4,The Organisation for Economic Cooperation and ...
5,It is important to have an objective before at...
6,Mere copy paste or hyperlinking could also be...
7,Usergenerated content occurs when a product cu...
8,According to Cisco Systems in 2016 an average ...
9,Reaching 166 billion daily active users in Q4 ...



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [21]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Coding_(social_sciences)', 1, 'assigned labels (sometimes called codes)'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Quantitative_research', 1, 'quantitatively'), ('https://en.wikipedia.org/wiki/Statistics', 1, 'statistical methods'), ('https://en.wikipedia.org/wiki/Qualitative_research', 1, 'qualitative'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning')]


In [22]:
print(contentPTags)

[<p><b>Content analysis</b> is the study of <a href="/wiki/Document" title="Document">documents</a> and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.
</p>, <p>Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of <a href="/wiki/Text_(literary_theory)" title="Text (literary theory)">texts</a> or artifacts which are <a href="/wiki/Coding_(social_sciences)" title="Coding (social sciences)">assigned labels (sometimes called codes)</a> to indicate the presence of interesting, <a href="/wiki/Semantic

Another excursion: Why do we use enumerate() here? enumerate() takes a collection, enumerates, and returns an enumate object with both the numbers and the collection. For example, contentPTags (the collection we used here) is comprised of paragraphs. We want the paragraph number of each paragraph. And this is what enumerate() does: it returns the paragraph number and the paragraph. 

We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [23]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,Content analysis is the study of documents and...,https://en.wikipedia.org/wiki/Content_analysis,0
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4
5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,5
6,Quantitative content analysis highlights frequ...,https://en.wikipedia.org/wiki/Content_analysis,6
7,Siegfried Kracauer provides a critique of quan...,https://en.wikipedia.org/wiki/Content_analysis,7
8,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,8
9,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [24]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 
                'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [25]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,Content analysis is the study of documents and...,https://en.wikipedia.org/wiki/Content_analysis,None,None
1,1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,None,None
2,2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,None,None
3,3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,None,None
4,4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,None,None
5,5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
6,6,Quantitative content analysis highlights frequ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
7,7,Siegfried Kracauer provides a critique of quan...,https://en.wikipedia.org/wiki/Content_analysis,None,None
8,8,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,None,None
9,9,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,None,None



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



In [26]:
# Extract relative links
UGC_otherPageURLS = []
for paragraphNum, pTag in enumerate(UGCPTags):
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        relurl = aTag.get('href')
        linkText = aTag.text
        UGC_otherPageURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(UGC_otherPageURLS[:10])

[('https://en.wikipedia.org/wiki/User_(computing)', 1, 'users'), ('https://en.wikipedia.org/wiki/Social_media', 1, 'social media'), ('https://en.wikipedia.org/wiki/Wiki', 1, 'wikis'), ('https://en.wikipedia.org/wiki/Flat_organization', 2, 'flattening'), ('https://en.wikipedia.org/wiki/BBC', 2, 'BBC'), ('https://en.wikipedia.org/wiki/Time_(magazine)', 2, 'TIME Magazine'), ('https://en.wikipedia.org/wiki/Web_2.0', 2, 'Web 2.0'), ('https://en.wikipedia.org/wiki/CNN', 2, 'CNN'), ('https://en.wikipedia.org/wiki/Amateur', 3, 'amateurs'), ('https://en.wikipedia.org/wiki/Mass_media', 3, "'packaged goods media' of the past century")]


In [27]:
# Construct dataframe
UGCParagraphsDF['source'] = [wikipedia_UGC] * len(UGCParagraphsDF['paragraph-text'])
UGCParagraphsDF['paragraph-number'] = range(len(UGCParagraphsDF['paragraph-text']))

UGCParagraphsDF

,paragraph-text,source,paragraph-number
0,Usergenerated content UGC alternatively known ...,https://en.wikipedia.org/wiki/User-generated_c...,0
1,Usergenerated content is used for a wide range...,https://en.wikipedia.org/wiki/User-generated_c...,1
2,The advent of usergenerated content marked a s...,https://en.wikipedia.org/wiki/User-generated_c...,2
3,The role of the passive audience therefore has...,https://en.wikipedia.org/wiki/User-generated_c...,3
4,The Organisation for Economic Cooperation and ...,https://en.wikipedia.org/wiki/User-generated_c...,4
5,It is important to have an objective before at...,https://en.wikipedia.org/wiki/User-generated_c...,5
6,Mere copy paste or hyperlinking could also be...,https://en.wikipedia.org/wiki/User-generated_c...,6
7,Usergenerated content occurs when a product cu...,https://en.wikipedia.org/wiki/User-generated_c...,7
8,According to Cisco Systems in 2016 an average ...,https://en.wikipedia.org/wiki/User-generated_c...,8
9,Reaching 166 billion daily active users in Q4 ...,https://en.wikipedia.org/wiki/User-generated_c...,9


In [28]:
UGCParagraphsDF['source-paragraph-number'] = [None] * len(UGCParagraphsDF['paragraph-text'])
UGCParagraphsDF['source-paragraph-text'] = [None] * len(UGCParagraphsDF['paragraph-text'])

for urlTuple in UGC_otherPageURLS[:3]:
    UGCParagraphsDF = UGCParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)

UGCParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,Usergenerated content UGC alternatively known ...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
1,1,Usergenerated content is used for a wide range...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
2,2,The advent of usergenerated content marked a s...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
3,3,The role of the passive audience therefore has...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
4,4,The Organisation for Economic Cooperation and ...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
5,5,It is important to have an objective before at...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
6,6,Mere copy paste or hyperlinking could also be...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
7,7,Usergenerated content occurs when a product cu...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
8,8,According to Cisco Systems in 2016 an average ...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
9,9,Reaching 166 billion daily active users in Q4 ...,https://en.wikipedia.org/wiki/User-generated_c...,None,None


In [29]:
# Set number of links to extract from one site
num_links = 5

# Recursive Spider function 
def spider_recursive(targetURL, sourceParNum, sourceText, levels, stop_level):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    otherPageURLS = []
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
        tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
        for aTag in tagLinks:
            relurl = aTag.get('href')
            linkText = aTag.text
            otherPageURLS.append((
                urllib.parse.urljoin(wikipedia_base_url, relurl),
                parNum,
                linkText,
                levels + 1,
                stop_level
            ))
    if levels == stop_level:
        return pandas.DataFrame(parsDict)
    for urlTuple in otherPageURLS[:num_links]:
        parsDF = pandas.DataFrame(parsDict).append(spider_recursive(*urlTuple), ignore_index=True)
    return parsDF

In [30]:
# Recursively extract links from paragraphs starting on wikipedia propaganda site
# Stop spidering when reach to 2 levels
# Only 5 links will be extracted from one site for efficiency reason
spider_recursive(wikipedia_UGC, None, None, 0, 2)

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,\n,https://en.wikipedia.org/wiki/User-generated_c...,None,None
1,1,"User-generated content (UGC), alternatively kn...",https://en.wikipedia.org/wiki/User-generated_c...,None,None
2,2,User-generated content is used for a wide rang...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
3,3,The advent of user-generated content marked a ...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
4,4,The role of the passive audience therefore has...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
5,5,The Organisation for Economic Co-operation and...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
6,6,It is important to have an objective before at...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
7,7,Mere copy & paste or hyperlinking could also b...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
8,8,User-generated content occurs when a product's...,https://en.wikipedia.org/wiki/User-generated_c...,None,None
9,9,"According to Cisco Systems, in 2016 an average...",https://en.wikipedia.org/wiki/User-generated_c...,None,None


This recursive spidering method extracts 5 links from each url and stops when it reaches to 2nd level of links. We can change the parameters passed in the method to change the number of links and the level we extract from one target url.

## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [31]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":3707,"posts-type":false,"posts":[{"id":"639732477227892736","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/639732477227892736","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/639732477227892736\/cat-demands-hairspray-sniffs","type":"photo","date-gmt":"2021-01-08 09:00:32 GMT","date":"Fri, 08 Jan 2021 10:00:32","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1610096432,"format":"html","reblog-key":"ZbrAwRC4","slug":"cat-demands-hairspray-sniffs","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"639732477227892736\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_639732477227892736\"><iframe id=\"like_iframe_639732477227892736\" src=\"https:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=66c22ab5319d742bca5762

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [32]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [33]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [34]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,date,id,photo-type,photo-url,tags
0,"Fri, 08 Jan 2021 10:00:32",639732477227892736,jpg,https://64.media.tumblr.com/31e8882b2fd934f7b9...,"[cat, cats, lol, lolcat, lolcats]"
1,"Fri, 08 Jan 2021 08:00:25",639724919901585408,jpg,https://64.media.tumblr.com/fb868dea41853ad734...,"[cat, cats, lol, lolcat, lolcats]"
2,"Fri, 08 Jan 2021 06:00:33",639717378220015616,jpg,https://64.media.tumblr.com/61e3d59b36423880a9...,"[cat, cats, lol, lolcat, lolcats]"
3,"Fri, 08 Jan 2021 04:01:19",639709877361688576,jpg,https://64.media.tumblr.com/12f20ceaaa85f5a91e...,"[cat, cats, lol, lolcat, lolcats]"
4,"Fri, 08 Jan 2021 02:01:06",639702313021440000,jpg,https://64.media.tumblr.com/2386aff8793f75dcd7...,"[cat, cats, lol, lolcat, lolcats]"
5,"Thu, 07 Jan 2021 22:00:33",639687179569889280,jpg,https://64.media.tumblr.com/61e3d59b36423880a9...,"[cat, cats, lol, lolcat, lolcats]"
6,"Thu, 07 Jan 2021 20:00:42",639679639402479617,jpg,https://64.media.tumblr.com/61e3d59b36423880a9...,"[cat, cats, lol, lolcat, lolcats]"
7,"Mon, 21 Dec 2020 10:00:25",638101724326936576,png,https://64.media.tumblr.com/653968fb91a4356937...,"[cat, cats, lol, lolcat, lolcats]"
8,"Wed, 09 Dec 2020 10:00:43",637014579763871744,png,https://64.media.tumblr.com/e649f37db75d7901b9...,"[cat, cats, lol, lolcat, lolcats]"
9,"Wed, 09 Dec 2020 08:00:36",637007022068498432,jpg,https://64.media.tumblr.com/9e5288c65670863e89...,"[cat, cats, lol, lolcat, lolcats]"


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [35]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [36]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. Lets start by looking at the Shakespeare files in the `data` directory 

In [37]:
with open('../data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

, and Train.]

PUCK
  If we shadows have offended,
  Think but this,--and all is mended,--
  That you have but slumber'd here
  While these visions did appear.
  And this weak and idle theme,
  No more yielding but a dream,
  Gentles, do not reprehend;
  If you pardon, we will mend.
  And, as I am an honest Puck,
  If we have unearned luck
  Now to 'scape the serpent's tongue,
  We will make amends ere long;
  Else the Puck a liar call:
  So, good night unto you all.
  Give me your hands, if we be friends,
  And Robin shall restore amends.

[Exit.]





End of Project Gutenberg Etext of A Midsummer Night's Dream by Shakespeare
PG has multiple editions of William Shakespeare's Complete Works



By the way, depending on your working directory, you might get errors such as: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt.' Don't panic, it's nothing, just check your working directory. 

Then to load all the files in `./data/Shakespeare` we can use a for loop with `scandir`:

In [38]:
targetDir = '../data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [39]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

,text
julius_caesar.txt,"Dramatis Personae\n\n JULIUS CAESAR, Roman st..."
as_you_like_it.txt,AS YOU LIKE IT\n\nby William Shakespeare\n\n\n...
tempest.txt,"The Tempest\n\nActus primus, Scena prima.\n\nA..."
phoenix_and_the_turtle.txt,THE PHOENIX AND THE TURTLE\n\nby William Shake...
king_lear.txt,The Tragedie of King Lear\n\n\nActus Primus. S...
passionate_pilgrim.txt,THE PASSIONATE PILGRIM\n\nby William Shakespea...
cymbeline.txt,The Tragedie of Cymbeline\n\nActus Primus. Sco...
coriolanus.txt,THE TRAGEDY OF CORIOLANUS\n\nby William Shakes...
two_gentlemen_of_verona.txt,THE TWO GENTLEMEN OF VERONA\n\nby William Shak...
rape_of_lucrece.txt,THE RAPE OF LUCRECE\n\nby William Shakespeare\...


Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [40]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.3
%���������
4 0 obj
<< /Length 5 0 R /Filter /FlateDecode >>
stream
x�]۶�F�}�W�c����T���C�i�<t�bM�f
Tn�<3_��CDf��J�N�i��#�%.;.���	?߄��7�]8������ux��}޾m����y��bǾ����!���$�Ǯ���C��F�����p���5��1��1�P<�{�$��/$�P�s�v��PgH?�����Q�~�*�:l��ˇ�m�ǰ��C�l����܊��E�����!�^�y��m�$�Ý���wۡل׼�6w���ī�K�~؞���r��~?�ˡkO�;6IH�9{ԡ��� ]?�E�E��~���.l������+��W�_�����C��S�|�~C��N�3ӛB`8�ޚj9���AZ��0�d�l^������SY	�Ƨ��>q�ۇ&
����.�����0����; ��>a8�$w�p��p����ST���. �7��@����)��&1�|���WՃ jOv�G2b�L8I��N�@gǍ������O�C��������IN@@�����}�8��+L����a�&ү�o�V(���0���+5�
SfS&��<�2�����>l�V��&��=4⇤=�W��<�JMo����"����d�C����[vY�|K{_ܔ\����%H�/@'�QA�+D�l��c��L�G�.��	�̎�V�:f>���AwK���o$`D��bE45�0�%th6h�����>*�2vQd�+M��Y}�Q���u�[���N�o'b��/u�.r'Z���J�e8�v��;��{T�	�����^8�  l<�E�<���b�����C8j��f��xB>K�����|w��f�|?�s̭��Y�'�Ip&�"�A���f�?�!IYi���U�"��y;���#��e3)�+B�&����<E9I�g�/]"D��yfC;e����Y^�z ��s'�)/�X�-HY��<ˬ�ݰ

It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [41]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [42]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [43]:
print(readPDF(infoExtractionBytes)[:550])

Department of  Sociology 

THE UNIVERSITY OF CHICAGO 

SOCIOLOGY 40133 

Computational Content Analysis 

Friday 1:00 – 3:50pm 
Winter 2017-2018 
Classroom: Harper Memorial 130       
http://chalk.uchicago.edu/ 

 

                                                                                           

          Office: McGiffert 210 
                                                    Tel.: 834-3612; jevans@uchicago.edu 
                                  Office Hours: Thursday 12:30-2:30pm 

     

        James A. Evans            

    


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [44]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud_2020` so we can use it later without having to retype it.

In [45]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [46]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [47]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


In [48]:
survey_pdf = 'https://arxiv.org/pdf/1901.09659.pdf'
surveyRequest = requests.get(survey_pdf, stream=True)
print(surveyRequest.text[:1000])

%PDF-1.5
%����
341 0 obj
<< /Linearized 1 /L 1448784 /H [ 1090 425 ] /O 345 /E 80040 /N 39 /T 1446466 >>
endobj
                                                                                                       
342 0 obj
<< /Type /XRef /Length 63 /Filter /FlateDecode /DecodeParms << /Columns 5 /Predictor 12 >> /W [ 1 3 1 ] /Index [ 341 71 ] /Info 352 0 R /Root 343 0 R /Size 412 /Prev 1446467               /ID [<dfbd6377287a40d47707de118470f3d6><dfbd6377287a40d47707de118470f3d6>] >>
stream
x�cbd`�g`b``8	"YL�l0ɸL�\GAdU�d*������L��/��u10���#�	,
endstream
endobj
                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [50]:
surveyBytes = io.BytesIO(surveyRequest.content)
print(readPDF(surveyBytes)[:1000])

Response Retrieval Inspired by Recommendation Systems 

Simple Surveys:  

 
 

Nandana Sengupta 

Azim Premji University 

Nati Srebro 

 

 

Toyota Technological Institute of Chicago 

University of Chicago 

James Evans 

University of Chicago 

 
 
Abstract 
 
In the last decade, the use of simple rating and comparison surveys has proliferated on social and 
digital media platforms to fuel recommendations. These simple surveys and their extrapolation 
with machine learning algorithms like matrix factorization shed light on user preferences over 
large and growing pools of items, such as movies, songs and ads. Social scientists have a long 
history of measuring perceptions, preferences and opinions, often over smaller, discrete item sets 
with exhaustive rating or ranking surveys. This paper introduces simple surveys for social 
science application. We ran experiments to compare the predictive accuracy of both individual 
and aggregate comparative assessments using four types of si

In [51]:
surveyContent = readPDF(surveyBytes)
surveysentence = surveyContent.replace('\n','').split('.')
survey_df = pandas.DataFrame(surveysentence,columns=['content'])
survey_df

,content
0,Response Retrieval Inspired by Recommendation ...
1,These simple surveys and their extrapolation ...
2,Social scientists have a long history of meas...
3,This paper introduces simple surveys for soci...
4,We ran experiments to compare the predictive ...
5,"Across contexts, we find that continuous scal..."
6,Binary choice surveys are quick and perform b...
7,"Pairwise comparisons, by contrast, perform we..."
8,We also demonstrate how findings from these s...
9,We conclude by reflecting on differences betw...


The above codes extract text contents from the link of a pdf document. readPDF method is created above this section, which is based on code from http://stackoverflow.com/a/20905381/4955164.
Now we extract and organize textual content into a pandas dataframe for further analysis.

### Other sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

